# Libraries

In [14]:
!pip install geemap
!pip install earthengine-api



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# API Call for Google Earth Engine

In [1]:

import ee
ee.Authenticate()
print("Successfull Authentication")
ee.Initialize(project='mygeo-475717')
print("Successfull Intitiliazation")




Successfull Authentication
Successfull Intitiliazation


# Google Map Displayer

In [2]:
import geemap
import ee
ee.Initialize(project='mygeo-475717')
Map = geemap.Map()

countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")
greece = countries.filter(ee.Filter.eq('country_na', 'Greece'))   

Map.addLayer(greece,{},'Greece')
Map


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

# Dataset Example

In [11]:
import ee
import geemap
ee.Initialize(project='mygeo-475717')
print("Successfull Intitiliazation")
Map=geemap.Map()

countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")
greece = countries.filter(ee.Filter.eq('country_na', 'Greece'))   

dataset = (
    ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
    .filterDate('2022-06-01', '2022-09-01')
    .filterBounds(greece)  
)

Map.addLayer(dataset,{},'Greece')
Map


Successfull Intitiliazation


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

# FIRMS

In [26]:

import ee
import geemap

ee.Initialize(project='mygeo-475717')
Map = geemap.Map()


Land = (
    ee.ImageCollection('LANDSAT/LC09/C02/T1')
    .filterDate('2022-01-01', '2022-12-01')
)

Fire = (
    ee.ImageCollection('FIRMS')
    .filterDate('2020-01-01', '2020-12-31')
)


Fires= Fire.select('confidence').max().toFloat()
Fires1= Fire.select('T21').max().toFloat()
Lands= Land.select(['B4', 'B3', 'B2']).median().toFloat()  
#Visualize for map
T21Vis= {'min': 300, 'max': 500, 'palette': ['yellow', 'orange', 'red']}
ConfidenceVis = {'min': 0, 'max': 100, 'palette': ['yellow', 'orange', 'red']}
LandVis= { 'min':0.0 , 'max':30000}

#Masking 
mask=Fires.gt(0)
mask1=Fires1.gt(0)
Fires_masked = Fires.updateMask(mask)
Fires1_masked = Fires1.updateMask(mask1)


Map.addLayer(Lands, LandVis, 'Landsat')
Map.addLayer(Fires1_masked,T21Vis, 'T21')
Map.addLayer(Fires_masked, ConfidenceVis, 'confidence')

Map



Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

# FIRMS Downlaod and Preprocessing

In [33]:
import ee
import geemap

ee.Initialize(project='mygeo-475717')
Map = geemap.Map()


Land = (
    ee.ImageCollection('LANDSAT/LC09/C02/T1')
    .filterDate('2022-01-01', '2022-12-01')
)

Fire = (
    ee.ImageCollection('FIRMS')
    .filterDate('2020-01-01', '2020-12-31')
)


Fires= Fire.select('confidence').max().toFloat()
Fires1= Fire.select('T21').max().toFloat()
Lands= Land.select(['B4', 'B3', 'B2']).median().toFloat()  
#Visualize for map
T21Vis= {'min': 300, 'max': 500, 'palette': ['yellow', 'orange', 'red']}
ConfidenceVis = {'min': 0, 'max': 100, 'palette': ['yellow', 'orange', 'red']}
LandVis= { 'min':0.0 , 'max':30000}

#Masking 
mask=Fires.gt(0)
mask1=Fires1.gt(0)
Fires_masked = Fires.updateMask(mask)
Fires1_masked = Fires1.updateMask(mask1)


Map.addLayer(Lands, LandVis, 'Landsat')
Map.addLayer(Fires1_masked,T21Vis, 'T21')
Map.addLayer(Fires_masked, ConfidenceVis, 'confidence')

Map


#Download  etc..

combined = Lands.addBands(Fires_masked).addBands(Fires1_masked)
#Each band converts to unit8 (0-255) Scaling and Normalizing
def scale_band(band):
    band_min = ee.Image.constant(
        band.reduceRegion(
            reducer=ee.Reducer.min(),
            geometry=greece.geometry(),
            scale=500
        ).values().get(0)
    )
    
    band_max = ee.Image.constant(
        band.reduceRegion(
            reducer=ee.Reducer.max(),
            geometry=greece.geometry(),
            scale=500
        ).values().get(0)
    )
    
    return band.subtract(band_min) \
               .divide(band_max.subtract(band_min)) \
               .multiply(255) \
               .toUint8()

bands_to_scale = {
    'landsat_rgb': ['B4','B3','B2'],
    'firms_T21': ['T21'],
    'firms_confidence': ['confidence']
}


scaled_landsat_bands = [scale_band(combined.select(b)) for b in bands_to_scale['landsat_rgb']]
landsat_uint8 = ee.Image.cat(scaled_landsat_bands)


scaled_T21 = [scale_band(combined.select(b)) for b in bands_to_scale['firms_T21']]
T21_uint8 = ee.Image.cat(scaled_T21)

scaled_conf = [scale_band(combined.select(b)) for b in bands_to_scale['firms_confidence']]
conf_uint8 = ee.Image.cat(scaled_conf)


all_bands = {
    'landsat': ['B4','B3','B2'],
    'firms_T21': ['T21'],
    'firms_confidence': ['confidence']
}

scaled_images = []

for dataset_name, band_list in all_bands.items():
    for b in band_list:
        band = combined.select(b)        
        scaled_band = scale_band(band)  
        scaled_images.append(scaled_band)

#Combine all images
combined_uint8 = ee.Image.cat(scaled_images)

#Download  
task = ee.batch.Export.image.toDrive(
    image=combined_uint8 ,
    description='FIRMS_Confidence_2020_Greece',
    folder='GEE_exports',
    fileNamePrefix='FIRMS_2020_Greece',
    region=greece.geometry(),  
    scale=500,
    maxPixels=1e13
)

task.start()

# Rasterio Display for Geotiff images 
Χρησιμοποιείται ο παρακάτω κώδικας:

import rasterio
import rasterio.plot

data_name = "Firms.tif"
tiff = rasterio.open(data_name)
rasterio.plot.show(tiff, title = "Example")


1.Rasterio plot
![1](Figure_1.png)
2.Μέσω Google earth engine
![2](firms.png)
3.Google map displayer with 2 datasets combined FIRMS + Landsat 
![3](example.png)

